In [ ]:
import os
import json
import boto3
 
# === Configure AWS Bedrock ===
bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")
 
MODEL_ID = ""  # your custom model ARN
 
 
def call_model(prompt: str) -> str:
    """Send prompt to Bedrock model and return response text."""
    response = bedrock.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            "messages": [
                {"role": "user", "content": [{"text": prompt}]}
            ]
        }),
        contentType="application/json",
        accept="application/json"
    )
 
    # Parse response
    result = json.loads(response["body"].read())
    output = ""
 
    if "outputText" in result:
        output = result["outputText"]
    elif "output" in result and "message" in result["output"]:
        content = result["output"]["message"].get("content", [])
        if content and "text" in content[0]:
            output = content[0]["text"]
 
    return output.strip()
 
 
def process_prompts(input_dir: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)
 
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(".json"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
 
            try:
                with open(input_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
 
                # Handle dict or list
                if isinstance(data, dict):
                    prompt = data.get("prompt", "")
                elif isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
                    prompt = data[0].get("prompt", "")
                else:
                    print(f"⚠️ Skipping {filename}, invalid JSON structure.")
                    continue
 
                if not prompt.strip():
                    print(f"⚠️ Skipping {filename}, no prompt found.")
                    continue
 
                print(f"➡️ Sending {filename} to model...")
                output_text = call_model(prompt)
 
                # Save output JSON with same filename
                with open(output_path, "w", encoding="utf-8") as f:
                    json.dump({"output": output_text}, f, indent=2, ensure_ascii=False)
 
                print(f"✅ Saved output to {output_path}")
 
            except Exception as e:
                print(f"❌ Error processing {filename}: {e}")
 
 
if __name__ == "__main__":
    input_dir = r"C:\Users\Administrator\Desktop\finetuning_20\email_prompts_json"  # folder with prompt JSONs
    output_dir = r"C:\Users\Administrator\Desktop\20-new-op1"         # folder to save model outputs
    process_prompts(input_dir, output_dir)